In [0]:
#Zadanie 2

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

spark = SparkSession.builder.appName("DiamondsCSV").getOrCreate()

#definiownie schematu danych
schema = StructType([
    StructField("carat", DoubleType(), True),
    StructField("cut", StringType(), True),
    StructField("color", StringType(), True),
    StructField("clarity", StringType(), True),
    StructField("depth", DoubleType(), True),
    StructField("table", DoubleType(), True),
    StructField("price", IntegerType(), True),
    StructField("x", DoubleType(), True),
    StructField("y", DoubleType(), True),
    StructField("z", DoubleType(), True)
])

#zaladowalam plik csv do zakladki Catalogs jako table i wczytalam
df = spark.read.table("default.diamonds_1_csv")
df.show(10)



+----+-----+---------+-----+-------+-----+-----+-----+----+----+----+
| _c0|  _c1|      _c2|  _c3|    _c4|  _c5|  _c6|  _c7| _c8| _c9|_c10|
+----+-----+---------+-----+-------+-----+-----+-----+----+----+----+
|null|carat|      cut|color|clarity|depth|table|price|   x|   y|   z|
|   1| 0.23|    Ideal|    E|    SI2| 61.5|   55|  326|3.95|3.98|2.43|
|   2| 0.21|  Premium|    E|    SI1| 59.8|   61|  326|3.89|3.84|2.31|
|   3| 0.23|     Good|    E|    VS1| 56.9|   65|  327|4.05|4.07|2.31|
|   4| 0.29|  Premium|    I|    VS2| 62.4|   58|  334| 4.2|4.23|2.63|
|   5| 0.31|     Good|    J|    SI2| 63.3|   58|  335|4.34|4.35|2.75|
|   6| 0.24|Very Good|    J|   VVS2| 62.8|   57|  336|3.94|3.96|2.48|
|   7| 0.24|Very Good|    I|   VVS1| 62.3|   57|  336|3.95|3.98|2.47|
|   8| 0.26|Very Good|    H|    SI1| 61.9|   55|  337|4.07|4.11|2.53|
|   9| 0.22|     Fair|    E|    VS2| 65.1|   61|  337|3.87|3.78|2.49|
+----+-----+---------+-----+-------+-----+-----+-----+----+----+----+
only showing top 10 

In [0]:
#zadanie 3
import pandas as pd
from pyspark.sql import SparkSession

df_broken_csv = spark.sql("SELECT * FROM diamonds_broken_csv")
df_broken_csv.show()


# uszkodzonenie pliku
data = [
    ["carat", "cut", "color", "clarity", "depth", "table", "price", "x", "y", "z"], 
    [0.23, "Ideal", "E", "SI2", 61.5, 55, 326, 3.95, 3.98, 2.43],  # OK
    [None, "Premium", "E", "SI1", 59.8, 61, 326, 3.89, 3.84, 2.31],  # zla wartosc jedna
    [0.31, "Good", "D", "VS1", 58.1, 57, None, 4.34, 4.35, 2.75],  # brak wartosci jednej
]

df_broken = pd.DataFrame(data[1:], columns=data[0])
df_spark = spark.createDataFrame(df_broken)

#tymczasowa tabela - problem ze stworzenie jakiejs na stale..
#df_spark.createOrReplaceTempView("diamonds_broken")

df_spark.write.mode("overwrite").option("header", True).csv("dbfs:/Workspace/diamonds_broken.csv")


# PERMISSIVE (domyślny) błędne wartości jako NULL
df_permissive = spark.sql("SELECT * FROM diamonds_broken")
print("PERMISSIVE mode:")
df_permissive.show()

# DROPMALFORMED – usuwanie błędnych wierszy 
df_dropmalformed = df_permissive.dropna()
print("DROPMALFORMED mode:")
df_dropmalformed.show()

# FAILFAST jeśli są jakiekolwiek błędne wartości, to wyjątek
try:
    if df_permissive.where("carat IS NULL OR carat = '?' OR price IS NULL").count():
        raise ValueError("FAILFAST mode error: Niepoprawne dane")
    print("FAILFAST mode:")
    df_permissive.show()
except ValueError as e:
    print(e)



+----+-----+---------+-----+-------+-----+-----+-----+----+----+----+
| _c0|  _c1|      _c2|  _c3|    _c4|  _c5|  _c6|  _c7| _c8| _c9|_c10|
+----+-----+---------+-----+-------+-----+-----+-----+----+----+----+
|null|carat|      cut|color|clarity|depth|table|price|   x|   y|   z|
|   1| 0.23|    Ideal|    E|    SI2| 61.5|   55|  326|3.95|3.98|2.43|
|   2| 0.21|  Premium|    E|    SI1| 59.8|   61|  326|3.89|3.84|2.31|
|   3| 0.23|     Good|    E|    VS1| 56.9|   65|  327|4.05|4.07|2.31|
|   4| 0.29|  Premium|    I|    VS2| 62.4|   58|  334| 4.2|4.23|2.63|
|   5| 0.31|     Good|    J|    SI2| 63.3|   58|  335|4.34|4.35|2.75|
|   6| 0.24|Very Good|    J|   VVS2| 62.8|   57|  336|3.94|3.96|2.48|
|   7| 0.24|Very Good|    I|   VVS1| 62.3|   57|  336|3.95|3.98|2.47|
|   8| 0.26|Very Good|    H|    SI1| 61.9|   55|  337|4.07|4.11|2.53|
|   9| 0.22|     Fair|    E|    VS2| 65.1|   61|  337|3.87|3.78|2.49|
|  10| 0.23|Very Good|    H|    VS1| 59.4|   61|  338|   4|4.05|2.39|
|  11|  0.3|     Goo

In [0]:
# ZADANIE 4

df_spark.show()

#zapis do .parquet
df_spark.write.mode("overwrite").parquet("dbfs:/Workspace/diamonds.parquet")
df_parquet = spark.read.parquet("dbfs:/Workspace/diamonds.parquet")
df_parquet.show()

#zapis do .json
df_spark.write.mode("overwrite").json("dbfs:/Workspace/diamonds1.json")
df_json1 = spark.read.json("dbfs:/Workspace/diamonds1.json")
df_json1.show()


+-----+-------+-----+-------+-----+-----+-----+----+----+----+
|carat|    cut|color|clarity|depth|table|price|   x|   y|   z|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+
| 0.23|  Ideal|    E|    SI2| 61.5|   55|326.0|3.95|3.98|2.43|
| null|Premium|    E|    SI1| 59.8|   61|326.0|3.89|3.84|2.31|
| 0.31|   Good|    D|    VS1| 58.1|   57| null|4.34|4.35|2.75|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+

+-----+-------+-----+-------+-----+-----+-----+----+----+----+
|carat|    cut|color|clarity|depth|table|price|   x|   y|   z|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+
| 0.23|  Ideal|    E|    SI2| 61.5|   55|326.0|3.95|3.98|2.43|
| null|Premium|    E|    SI1| 59.8|   61|326.0|3.89|3.84|2.31|
| 0.31|   Good|    D|    VS1| 58.1|   57| null|4.34|4.35|2.75|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+

+-----+-------+-----+-------+-----+-----+-----+----+----+----+
|carat|clarity|color|    cut|depth|price|table|   x| 